In [1]:
from datetime import datetime, timedelta
import dateutil.parser
import time
from os import walk
import pandas as pd
import re

import tushare as ts
import asyncio
from tqdm.asyncio import tqdm

In [22]:
ts.set_token('e15be55df8e1570bf0b7bd6f56c50111d19109fdad5abcb05091579b')
pro = ts.pro_api('e15be55df8e1570bf0b7bd6f56c50111d19109fdad5abcb05091579b')

recrawl = False

# 股票列表 + 上市公司基本信息

In [5]:
%%time

if not recrawl:
    stock_info_list = pd.read_csv('stock_list.csv')
else:
    stock_info_list = pro.stock_basic().merge(pro.stock_company(), on='ts_code')
    stock_info_list.to_csv('stock_list.csv', index=False, encoding='utf-8')
    
display(stock_info_list)

,ts_code,symbol,name,area,industry,market,list_date,exchange,chairman,manager,secretary,reg_capital,setup_date,province,city,website,email,employees
0,000001.SZ,1,平安银行,深圳,银行,主板,19910403,SZSE,谢永林,胡跃飞,周强,1.940592e+06,19871222,广东,深圳市,bank.pingan.com,PAB_db@pingan.com.cn,36115.0
1,000002.SZ,2,万科A,深圳,全国地产,主板,19910129,SZSE,郁亮,祝九胜,朱旭,1.161773e+06,19840530,广东,深圳市,www.vanke.com,ir@vanke.com;peterluk@vanke.com,133455.0
2,000004.SZ,4,国华网安,深圳,互联网,主板,19910114,SZSE,黄翔,黄翔,黄冰夏,1.650526e+04,19860505,广东,深圳市,www.sz000004.cn,gnkjsz@163.com;jyz6187@qq.com;xuliruan@hotmail...,251.0
3,000005.SZ,5,世纪星源,深圳,环境保护,主板,19901210,SZSE,丁芃,郑列列,罗晓春,1.058537e+05,19890508,广东,深圳市,www.fountain.com.cn,fountain@sfc.com.cn;xiaochun@sfc.com.cn,680.0
4,000006.SZ,6,深振业A,深圳,区域地产,主板,19920427,SZSE,赵宏伟,李伟,杨晓东,1.349995e+05,19890525,广东,深圳市,www.zhenye.com,szzygp@126.com,385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,688699.SH,688699,明微电子,深圳,半导体,科创板,20201218,SSE,王乐康,李照华,郭王洁,7.436800e+03,20031030,广东,深圳市,www.chinaasic.com,ir@chinaasic.com,385.0
4197,688777.SH,688777,中控技术,浙江,软件服务,科创板,20201124,SSE,CUI SHAN,CUI SHAN,莫威,4.940840e+04,19991207,浙江,杭州市,www.supcontech.com,ir@supcon.com,3607.0
4198,688788.SH,688788,科思科技,深圳,通信设备,科创板,20201022,SSE,刘建德,刘建德,庄丽华,7.553423e+03,20040227,广东,深圳市,www.consys.com.cn,securities@consys.com.cn,487.0
4199,688819.SH,688819,天能股份,浙江,电气设备,科创板,20210118,SSE,杨建芬,杨建芬,胡敏翔,9.721000e+04,20030313,浙江,湖州市,www.cn-tn.com,dshbgs@tiannenggroup.com,22420.0


CPU times: user 60.1 ms, sys: 5.61 ms, total: 65.7 ms
Wall time: 64.7 ms


# A股后复权行情

In [6]:
async def get_daily_adjusted_stock_price_first(code):
    return None if code == '' else ts.pro_bar(ts_code=code, adj='hfq', start_date='19900101', end_date='20071231')
async def get_daily_adjusted_stock_price_second(code):
    return None if code == '' else ts.pro_bar(ts_code=code, adj='hfq', start_date='20080101', end_date='20210131')

if not recrawl:
    stock_price_post_adjusted = pd.read_feather('stock_price_post_adjusted.ftr')
else:
    stock_price_post_adjusted = []
    request_per_minute = 400
    for i in range(0, len(stock_info_list.ts_code), request_per_minute):
        st = time.time()
        aync_tasks = [asyncio.create_task(get_daily_adjusted_stock_price_first(code)) for code in stock_info_list.ts_code[i:i+request_per_minute]]
        stock_price_post_adjusted.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))
        
        st = time.time()
        aync_tasks = [asyncio.create_task(get_daily_adjusted_stock_price_second(code)) for code in stock_info_list.ts_code[i:i+request_per_minute]]
        stock_price_post_adjusted.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))

    stock_price_post_adjusted = pd.concat(stock_price_post_adjusted, ignore_index=True)
    stock_price_post_adjusted['trade_date'] = pd.to_datetime(stock_price_post_adjusted['trade_date'], format='%Y%m%d')
    stock_price_post_adjusted = stock_price_post_adjusted.sort_values(by=['ts_code', 'trade_date']).reset_index(drop=True)
    stock_price_post_adjusted.to_feather('stock_price_post_adjusted.ftr')
    
display(stock_price_post_adjusted)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,1991-04-04,48.76,48.76,48.76,48.76,49.00,-0.24,-0.4898,3.00,15.000
1,000001.SZ,1991-04-05,48.52,48.52,48.52,48.52,48.76,-0.24,-0.4922,2.00,10.000
2,000001.SZ,1991-04-08,48.04,48.04,48.04,48.04,48.52,-0.48,-0.9893,2.00,10.000
3,000001.SZ,1991-04-09,47.80,47.80,47.80,47.80,48.04,-0.24,-0.4996,4.00,19.000
4,000001.SZ,1991-04-10,47.56,47.56,47.56,47.56,47.80,-0.24,-0.5021,15.00,71.000
...,...,...,...,...,...,...,...,...,...,...,...
10780853,688981.SH,2021-01-25,56.64,58.58,55.82,58.02,57.15,0.87,1.5223,361339.68,2082488.409
10780854,688981.SH,2021-01-26,58.04,59.78,57.80,59.78,58.02,1.76,3.0334,542458.15,3199844.123
10780855,688981.SH,2021-01-27,59.80,59.95,57.50,57.72,59.78,-2.06,-3.4460,337522.20,1967491.697
10780856,688981.SH,2021-01-28,56.68,57.69,56.03,56.04,57.72,-1.68,-2.9106,256341.91,1455062.584


# 交易日历

In [7]:
if not recrawl:
    trade_cal = pd.read_csv('trade_cal.csv')
else:
    trade_cal = pro.trade_cal(exchange='', start_date='19900101', end_date='20210131').drop(columns=['exchange'])
    trade_cal.to_csv('trade_cal.csv')
    
display(trade_cal)

,Unnamed: 0,cal_date,is_open
0,0,19901219,1
1,1,19901220,1
2,2,19901221,1
3,3,19901222,0
4,4,19901223,0
...,...,...,...
10997,10997,20210127,1
10998,10998,20210128,1
10999,10999,20210129,1
11000,11000,20210130,0


# 上市公司管理层 + 管理层薪酬和持股

In [8]:
async def get_stk_managers(code):
    return pro.stk_managers(ts_code=code) if code != '' else None

if not recrawl:
    stk_managers_list = pd.read_csv('stk_managers_list.csv')
else:
    stk_managers_list = []
    request_per_minute = 400
    number_of_stock = 5
    for i in range(0, len(stock_info_list.ts_code), request_per_minute*number_of_stock):
        st = time.time()
        
        aync_tasks = [asyncio.create_task(get_stk_managers(','.join(stock_info_list.ts_code[i+j:i+j+number_of_stock]))) for j in range(0, request_per_minute*number_of_stock, number_of_stock)]
        stk_managers_list.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute*number_of_stock}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))

    stk_managers_list = pd.concat(stk_managers_list, ignore_index=True)
    stk_managers_list.to_csv('stk_managers_list.csv', index=False, encoding='utf-8')
    
display(stk_managers_list)

,ts_code,ann_date,name,gender,lev,title,edu,national,birthday,begin_date,end_date
0,000001.SZ,20190604,姚贵平,M,高管,副行长,本科,中国,1961.0,20180815.0,20190604.0
1,000001.SZ,20190604,姚贵平,M,董事会成员,执行董事,本科,中国,1961.0,20170629.0,20190604.0
2,000001.SZ,20190604,姚贵平,M,委员会成员,战略委员会委员,本科,中国,1961.0,20180129.0,20190604.0
3,000001.SZ,20190309,吴鹏,M,高管,副行长,博士,中国,1965.0,20110817.0,20190309.0
4,000001.SZ,20190307,孙永桢,F,监事,职工监事,硕士,中国,1968.0,20181025.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
429929,688388.SH,20190706,廖平元,M,委员会成员,提名委员会委员,本科,中国,1974.0,20171222.0,NaN
429930,688388.SH,20180530,蔡杨媚,F,董事会成员,董事,本科,中国,198211.0,20140520.0,20180525.0
429931,688388.SH,20171206,陈欣汉,M,董事会成员,董事,专科,中国,196112.0,20140520.0,20171201.0
429932,688388.SH,20171206,林国玉,M,董事会成员,董事,本科,中国,196310.0,20160421.0,20171201.0


In [9]:
async def get_stk_rewards(code):
    return pro.stk_rewards(ts_code=code) if code != '' else None

if not recrawl:
    stk_managers_rewards_list = pd.read_csv('stk_managers_rewards_list.csv')
else:
    stk_managers_rewards_list = []
    request_per_minute = 300
    number_of_stock = 3
    for i in range(0, len(stock_info_list.ts_code), request_per_minute*number_of_stock):
        st = time.time()
        
        aync_tasks = [asyncio.create_task(get_stk_rewards(','.join(stock_info_list.ts_code[i+j:i+j+number_of_stock]))) for j in range(0, request_per_minute*number_of_stock, number_of_stock)]
        stk_managers_rewards_list.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute*number_of_stock}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))

    stk_managers_rewards_list = pd.concat(stk_managers_rewards_list, ignore_index=True)
    stk_managers_rewards_list = stk_managers_rewards_list.fillna(0).query('hold_vol > 0 or reward > 0')
    stk_managers_rewards_list.to_csv('stk_managers_rewards_list.csv', index=False, encoding='utf-8')
    
display(stk_managers_rewards_list)

,ts_code,ann_date,end_date,name,title,reward,hold_vol
0,000001.SZ,20200828,20200630,胡跃飞,"董事,行长",0.0,4104.0
1,000001.SZ,20200828,20200630,项有志,"董事,副行长,首席财务官",0.0,6000.0
2,000002.SZ,20200828,20200630,郁亮,董事会主席,0.0,7306245.0
3,000002.SZ,20200828,20200630,解冻,监事会主席,0.0,1490745.0
4,000002.SZ,20200828,20200630,阙东武,监事,0.0,60700.0
...,...,...,...,...,...,...,...
789985,688981.SH,20100426,20091231,陈立武,董事,8149.0,0.0
789986,688981.SH,20100426,20091231,杨雄哲,董事,8149.0,0.0
789987,688981.SH,20100426,20091231,江上舟,董事,8149.0,0.0
789988,688981.SH,20100426,20091231,张汝京,董事,320328.0,0.0


# 申万行业

In [10]:
%%time

if not recrawl:
    industry_list = pd.read_csv('industry_list.csv')
else:
    industry_list = pd.concat(
        [pro.index_classify(level=f'L{i+1}', fields=['index_code', 'industry_name', 'level', 'industry_code']) for i in range(3)]
    ).sort_values(by=['industry_code']).reset_index(drop=True)

    # There is no such data, need manual processing
    
    
    industry_list.to_csv('industry_list.csv', index=False, encoding='utf-8')
    
    
display(industry_list)

,index_code,industry_name,level,industry_code
0,801010.SI,农林牧渔,L1,110000
1,801016.SI,种植业,L2,110100
2,850111.SI,种子生产,L3,110101
3,850112.SI,粮食种植,L3,110102
4,850113.SI,其他种植业,L3,110103
...,...,...,...,...
354,852211.SI,通信运营,L3,730101
355,801102.SI,通信设备,L2,730200
356,851012.SI,终端设备,L3,730201
357,851013.SI,通信传输设备,L3,730202


CPU times: user 9.69 ms, sys: 1.85 ms, total: 11.5 ms
Wall time: 10.5 ms


In [11]:
async def get_industry_member(index):
    return pro.index_member(index_code=index)

if not recrawl:
    industry_member_list = pd.read_csv('industry_member_list.csv')
else:
    industry_member_list = [asyncio.create_task(get_industry_member(index)) for index in industry_list.index_code if 'SI' in index]
    industry_member_list = [await t for t in tqdm(asyncio.as_completed(industry_member_list), total=len(industry_member_list))]
    industry_member_list = pd.concat(industry_member_list, ignore_index=True)
    industry_member_list.drop(columns=['out_date']).to_csv('industry_member_list.csv', index=False, encoding='utf-8')
    
display(industry_member_list)

,index_code,con_code,in_date
0,801010.SI,002679.SZ,20120612
1,801010.SI,002696.SZ,20120912
2,801010.SI,300313.SZ,20120504
3,801010.SI,600467.SH,20040405
4,801010.SI,000702.SZ,19970318
...,...,...,...
11200,851014.SI,300603.SZ,20170209
11201,851014.SI,603322.SH,20160804
11202,851014.SI,000971.SZ,20160701
11203,851014.SI,300383.SZ,20180321


# 指数基本信息 + 指数日线行情

In [12]:
# if not recrawl:
#     index_list = pd.read_csv('index_list.csv')
# else:
#     index_list = [pro.index_basic(market=market) for market in ['MSCI', 'CSI', 'SSE', 'SZSE', 'CICC', 'SW', 'OTH']]
#     index_list = pd.concat(index_list, ignore_index=True)
#     index_list.to_csv('index_list.csv', index=False, encoding='utf-8')
index_list = pd.read_csv('index_list.csv')[['ts_code', 'name']]
display(index_list)

,ts_code,name
0,000001.SH,上证指数
1,000986.SH,全指能源
2,000987.SH,全指材料
3,000988.SH,全指工业(SH)
4,000989.SH,全指可选
5,000990.SH,全指消费
6,000991.SH,全指医药
7,000992.SH,全指金融
8,000993.SH,全指信息
9,000994.SH,全指电信(SH)


In [13]:
index_col = ['ts_code', 'industry_code']
index_dict = [
    ['000986.SH', 210000], ['000986.SH', 220000], ['000987.SH', 240000], ['000987.SH', 610000], ['000988.SH', 630000],
    ['000988.SH', 640000], ['000988.SH', 420000], ['000988.SH', 620000], ['000988.SH', 280000], ['000988.SH', 230000],
    ['000988.SH', 650000], ['000989.SH', 330000], ['000989.SH', 360000], ['000989.SH', 450000], ['000989.SH', 460000],
    ['000989.SH', 270000], ['000989.SH', 720000], ['000990.SH', 340000], ['000990.SH', 350000], ['000990.SH', 510000],
    ['000990.SH', 110000], ['000991.SH', 370000], ['000992.SH', 480000], ['000992.SH', 490000], ['000992.SH', 430000],
    ['000993.SH', 710000], ['000994.SH', 730000], ['000995.SH', 410000]
]

if not recrawl:
    index_industry_list = pd.read_csv('index_industry_list.csv')
else:
    index_industry_list = index_list.merge(pd.DataFrame(index_dict, columns=index_col)).merge(industry_list)
    index_industry_list.to_csv('index_industry_list.csv', index=False, encoding='utf-8')
index_industry_list

,ts_code,name,industry_code,index_code,industry_name,level
0,000986.SH,全指能源,210000,801020.SI,采掘,L1
1,000986.SH,全指能源,220000,801030.SI,化工,L1
2,000987.SH,全指材料,240000,801050.SI,有色金属,L1
3,000987.SH,全指材料,610000,801710.SI,建筑材料,L1
4,000988.SH,全指工业(SH),630000,801730.SI,电气设备,L1
5,000988.SH,全指工业(SH),640000,801890.SI,机械设备,L1
6,000988.SH,全指工业(SH),420000,801170.SI,交通运输,L1
7,000988.SH,全指工业(SH),620000,801720.SI,建筑装饰,L1
8,000988.SH,全指工业(SH),280000,801880.SI,汽车,L1
9,000988.SH,全指工业(SH),230000,801040.SI,钢铁,L1


In [14]:
async def get_index_daily(index):
    return pro.index_daily(ts_code=index)

if not recrawl:
    index_daily_list = pd.read_csv('index_daily_list.csv')
else:
    index_daily_list = [asyncio.create_task(get_index_daily(index)) for index in index_list.ts_code]
    index_daily_list = [await t for t in tqdm(asyncio.as_completed(index_daily_list), total=len(index_daily_list))]
    index_daily_list = pd.concat(index_daily_list, ignore_index=True)
    index_daily_list.to_csv('index_daily_list.csv', index=False, encoding='utf-8')
    
display(index_daily_list)

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000001.SH,20210312,3453.0777,3447.2033,3454.5064,3417.2389,3436.8313,16.2464,0.4727,3.580270e+08,3.819643e+08
1,000001.SH,20210311,3436.8313,3369.9024,3436.8313,3369.9024,3357.7372,79.0941,2.3556,3.221120e+08,3.698054e+08
2,000001.SH,20210310,3357.7372,3389.8388,3397.0251,3354.8230,3359.2920,-1.5548,-0.0463,2.997492e+08,3.395717e+08
3,000001.SH,20210309,3359.2920,3415.3391,3429.1503,3328.3119,3421.4141,-62.1221,-1.8157,4.076525e+08,4.561962e+08
4,000001.SH,20210308,3421.4141,3524.9830,3542.3039,3421.2153,3501.9867,-80.5726,-2.3008,3.870480e+08,4.538592e+08
...,...,...,...,...,...,...,...,...,...,...,...
45454,000995.SH,20050110,986.9190,975.3610,986.9190,973.2120,976.1320,10.7870,1.1051,5.109630e+05,3.339195e+05
45455,000995.SH,20050107,976.1320,977.7900,989.9160,974.7920,977.8530,-1.7210,-0.1760,5.042461e+05,3.224360e+05
45456,000995.SH,20050106,977.8530,991.5040,991.5040,974.9730,990.9040,-13.0510,-1.3171,4.913317e+05,3.192199e+05
45457,000995.SH,20050105,990.9040,983.5360,992.4170,980.3000,983.3940,7.5100,0.7637,5.464595e+05,3.587151e+05


In [15]:
index_daily_list.drop_duplicates('ts_code', keep='last')

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
7387,000001.SH,19901219,99.980,96.050,99.980,95.790,100.0,-0.020,-0.0200,1260.00,494.311
11321,000986.SH,20050104,965.419,985.442,985.442,962.784,1000.0,-34.581,-3.4581,748990.56,354648.324
15255,000987.SH,20050104,976.039,993.262,993.262,975.317,1000.0,-23.961,-2.3961,3097109.52,1806270.149
18773,000988.SH,20050104,986.118,995.924,995.924,984.309,1000.0,-13.882,-1.3882,1616664.76,1027625.200
22707,000989.SH,20050104,987.344,996.621,996.621,984.853,1000.0,-12.656,-1.2656,1835525.05,967051.675
26620,000990.SH,20050104,984.817,996.546,996.546,984.406,1000.0,-15.183,-1.5183,356334.10,212880.614
30554,000991.SH,20050104,982.280,997.552,997.552,981.672,1000.0,-17.720,-1.7720,319237.60,196535.534
34488,000992.SH,20050104,988.757,995.880,995.880,985.490,1000.0,-11.243,-1.1243,752567.88,448571.909
38422,000993.SH,20050104,996.872,995.373,1004.371,987.825,1000.0,-3.128,-0.3128,756016.15,567008.751
41940,000994.SH,20050104,991.244,997.018,997.018,984.489,1000.0,-8.756,-0.8756,724980.31,310998.911


# 同花顺板块指数

In [16]:
if not recrawl:
    concept_list = pd.read_csv('concept_list.csv')
else:
    concept_list = pro.ths_index(exchange='A', type='N', fields=['ts_code', 'name', 'list_date'])
    concept_list.to_csv('concept_list.csv', index=False, encoding='utf-8')
    
display(concept_list)

,ts_code,name,list_date
0,885835.TI,参股银行,20190416
1,885472.TI,上海自贸区,20130813
2,885788.TI,网络直播,20180312
3,885881.TI,云办公,20200203
4,885785.TI,小米概念,20180306
...,...,...,...
266,885566.TI,大飞机,20140519
267,885841.TI,草地贪夜蛾防治,20190517
268,885760.TI,装配式建筑,20170918
269,885909.TI,辅助生殖,20201023


In [17]:
async def get_concept_member(id):
    return pro.ths_member(ts_code=id)

if not recrawl:
    concept_member_list = pd.read_csv('concept_member_list.csv')
else:
    concept_member_list = []
    request_per_minute = 200
    for i in range(0, len(concept_list.ts_code), request_per_minute):
        st = time.time()
        aync_tasks = [asyncio.create_task(get_concept_member(code)) for code in concept_list.ts_code[i:i+request_per_minute]]
        concept_member_list.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        if i*request_per_minute < len(concept_list.ts_code):
            time.sleep(max(st+61-et, 0))

    concept_member_list = pd.concat(concept_member_list, ignore_index=True)
    concept_member_list.to_csv('concept_member_list.csv', index=False, encoding='utf-8')
    
display(concept_member_list)

,ts_code,code,name
0,885835.TI,000039.SZ,中集集团
1,885835.TI,000100.SZ,TCL科技
2,885835.TI,000158.SZ,常山北明
3,885835.TI,000416.SZ,民生控股
4,885835.TI,000419.SZ,通程控股
...,...,...,...
26425,885883.TI,300929.SZ,华骐环保
26426,885883.TI,600008.SH,首创股份
26427,885883.TI,600587.SH,新华医疗
26428,885883.TI,601330.SH,绿色动力


In [18]:
async def get_concept_daily(id):
    return pro.ths_daily(ts_code=id)

if not recrawl:
    concept_daily_list = pd.read_csv('concept_daily_list.csv')
else:
    concept_daily_list = []
    request_per_minute = 200
    for i in range(0, len(concept_list.ts_code), request_per_minute):
        st = time.time()
        aync_tasks = [asyncio.create_task(get_concept_daily(code)) for code in concept_list.ts_code[i:i+request_per_minute]]
        concept_daily_list.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        if i*request_per_minute < len(concept_list.ts_code):
            time.sleep(max(st+61-et, 0))

    concept_daily_list = pd.concat(concept_daily_list, ignore_index=True)
    concept_daily_list.to_csv('concept_daily_list.csv', index=False, encoding='utf-8')
    
display(concept_daily_list)

,ts_code,trade_date,close,open,high,low,pre_close,avg_price,change,pct_change,vol,turnover_rate
0,885835.TI,20210312,978.641,970.667,979.184,964.477,969.661,9.0742,8.980,0.9261,44136753.0,1.351399
1,885835.TI,20210311,969.661,951.620,969.854,947.863,951.379,8.8395,18.282,1.9216,37890184.0,1.160139
2,885835.TI,20210310,951.379,968.248,969.511,949.671,964.720,8.9486,-13.341,-1.3829,35835766.0,1.097236
3,885835.TI,20210309,964.720,983.743,984.671,949.027,985.433,9.0143,-20.713,-2.1019,49886790.0,1.527466
4,885835.TI,20210308,985.433,999.321,1005.455,985.433,993.207,9.6383,-7.774,-0.7827,43858140.0,1.342877
...,...,...,...,...,...,...,...,...,...,...,...,...
284438,885883.TI,20200213,1161.098,1101.634,1169.957,1076.685,1106.603,9.2105,54.495,4.9245,5116310.2,3.711904
284439,885883.TI,20200212,1106.603,1085.685,1115.384,1078.654,1094.648,10.4376,11.955,1.0921,3133260.5,2.273193
284440,885883.TI,20200211,1094.648,1138.459,1138.459,1092.017,1168.644,9.4165,-73.996,-6.3318,4878659.7,6.024470
284441,885883.TI,20200210,1168.644,1181.258,1204.387,1136.456,1100.063,9.8656,68.581,6.2343,6891017.6,10.890380


# 财经新闻信息

In [20]:
### Crawl the raw news ###
news_dict = {
    'sina': '2018-10-08 10:22:47',
    'wallstreetcn': '2018-09-28 05:59:27',
    '10jqka': '2018-10-24 02:58:00',
    'eastmoney': '2018-10-25 07:34:49',
    'yuncaijing': '2018-10-26 12:23:15'
}

def get_news(source, begin_date):
    count = 0
    end_date = '2021-01-31 23:59:59'
    begin_datetime = dateutil.parser.parse(begin_date)
    end_datetime = dateutil.parser.parse(end_date)
    while end_datetime > begin_datetime:
        st = time.time()
        news_list = []
        for _ in range(100):
            news = pro.news(src=source, start_date=begin_date, end_date=end_date)
            if len(news) == 0:
                break
            end_datetime = dateutil.parser.parse(news.tail(1)['datetime'].values[0]) - timedelta(seconds=1)
            end_date = end_datetime.strftime('%Y-%m-%d %H:%M:%S')
            news_list.append(news)
        
        if len(news_list) == 0:
            break
        news_list = pd.concat(news_list).sort_values(by=['datetime'])
        news_list.to_csv(f'news/news_{source}_{end_datetime.strftime("%Y_%m_%d_%H_%M_%S")}.csv', index=False, encoding='utf-8')
        
        count += 1
        et = time.time()
        print(f'finished {count} run with end_datetime {end_datetime.strftime("%Y-%m-%d")}. Waiting {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))

if not recrawl:
    news_list = pd.read_feather('news.ftr')
else:
    for source, begin_date in news_dict.items():
        get_news(source, begin_date)
        
    ### Read the news files ###
    _, _, news_files = next(walk('news/'))
    news_list = []
    for file in news_files:
        _news = pd.read_csv(f'news/{file}')
        _news['src'] = file[5:5+file[5:].find('_')]
        news_list.append(_news)
    news_list = pd.concat(news_list).sort_values(by=['datetime']).reset_index(drop=True)

    ### Preprocess the news ###
    news_list['content'] = news_list['content'].str.replace('\n','')
    news_list['content'] = news_list['content'].str.replace(r'【.*】','',regex=True)
    news_list['content'] = (news_list['title']+'。').fillna('') + news_list['content']
    news_list = news_list.drop(columns=['title'])
    news_list.to_feather('news.ftr')
    
display(news_list)

,datetime,content,src
0,2018-09-28 05:59:27,加拿大驻美国大使Macnaughton：认为在北美自由贸易协定（NAFTA）谈判上取得一些进...,wallstreetcn
1,2018-09-28 06:09:04,特斯拉盘后跌超13.5%。,wallstreetcn
2,2018-09-28 06:18:39,美国国会参议院共和党人们将在参议院听证会之后会见特朗普提名的最高法院大法官Kavanaugh...,wallstreetcn
3,2018-09-28 06:30:07,日内请重点关注（以下均为北京时间）：① 全天 沪港通及深港通-南向-休市、内盘期货夜市休市；...,wallstreetcn
4,2018-09-28 06:46:55,美国国会参议院司法委员会关于最高法院大法官Kavanaugh不当行为指控的听证会结束。,wallstreetcn
...,...,...,...
1289727,2021-01-31 23:31:40,比特币日内下破33000美元/枚，跌幅达3.86%。,sina
1289728,2021-01-31 23:34:36,以太坊日内跌超5%，现报1308.65美元/枚。,sina
1289729,2021-01-31 23:40:41,太魔幻！深圳6盘齐开 打新大军挤爆银行、挤崩登记系统。深圳楼市又出魔幻事件！近日，深圳集中推...,eastmoney
1289730,2021-01-31 23:51:04,比特币短线跌超400美元，现报 32718.8美元/枚。,sina


In [21]:
if not recrawl:
    selected_news = pd.read_feather('selected_news.ftr')
else:
    selected_news = [news_list[news_list.content.str.contains(stock)] for stock in tqdm(stock_info_list.name.str.replace('*', '', regex=False))]
    selected_news = pd.concat(selected_news).sort_values(by=['datetime']).drop_duplicates().reset_index(drop=True)
    selected_news.to_feather('selected_news.ftr')
    
display(selected_news)

,datetime,content,src
0,2018-09-28 14:54:00,对冲研投讯，自2018年9月28日零时起，安阳钢铁股份有限公司价格调整如下：1、带肋钢筋下调...,wallstreetcn
1,2018-09-28 15:01:13,A股收盘，区块链板块上涨0.29%。A股收盘，区块链板块上涨0.29%，银之杰涨停，信息发展...,wallstreetcn
2,2018-09-28 16:07:13,五洲新春：拟斥资5000万至9900万元回购股份，回购股份价格不超过15元/股，回购期限不超...,wallstreetcn
3,2018-09-28 16:10:44,川化股份：10月8日起公司股票更名为“川能动力”，证券代码“000155”保持不变。,wallstreetcn
4,2018-09-28 16:13:47,蓝思科技公告称，公司控股股东香港蓝思拟非公开发行可交换公司债券，并以其持有的公司A股股票作为...,wallstreetcn
...,...,...,...
336188,2021-01-31 21:43:00,"永兴材料：拟11.25亿元投建年产2万吨电池级碳酸锂项目及...。永兴材料公布，为贯彻公司""...",10jqka
336189,2021-01-31 21:43:50,三天调研人员超500人 7000亿电动车巨头火了！原因在这里。期间机构调研热度与前周基本持平...,eastmoney
336190,2021-01-31 21:58:41,免税行业获政策支持 将成海外消费回流重要渠道。中办、国办近日印发《建设高标准市场体系行动方案...,eastmoney
336191,2021-01-31 22:16:16,最多巨亏近36亿！上市27年老牌日化企业炸锅 减值损失50亿 市值仅18亿！两位前高管被立案...,eastmoney


# 前十大股东

In [23]:
stock_info_list = pd.read_csv('stock_list.csv', parse_dates=['list_date', 'setup_date'])
stock_info_list = stock_info_list.query('market == "主板" or market == "中小板"')  # Focus only on mainboard and sme, cover all three-level sectors
stock_info_list = stock_info_list.query('list_date < "2020-01-01"').reset_index(drop=True)  # Remove those got listed too late (171 of them)

In [24]:
async def get_stk_holders_list_17(code):
    return None if code == '' else pro.top10_holders(ts_code=code, start_date=f'20170101', end_date=f'20181231')
async def get_stk_holders_list_19(code):
    return None if code == '' else pro.top10_holders(ts_code=code, start_date=f'20190101', end_date=f'20210101')

if not recrawl:
    stk_holders_list = pd.read_csv('stk_holders_list.csv')
else:
    stk_holders_list = []
    request_per_minute = 300
    for i in range(0, len(stock_info_list.ts_code), request_per_minute):
        st = time.time()
        aync_tasks = [asyncio.create_task(get_stk_holders_list_17(code)) for code in stock_info_list.ts_code[i:i+request_per_minute]]
        stk_holders_list.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))
        
        st = time.time()
        aync_tasks = [asyncio.create_task(get_stk_holders_list_19(code)) for code in stock_info_list.ts_code[i:i+request_per_minute]]
        stk_holders_list.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))
        
    stk_holders_list = pd.concat(stk_holders_list, ignore_index=True)
    stk_holders_list.to_csv('stk_holders_list.csv')
    
display(stk_holders_list)

100%|██████████| 300/300 [00:33<00:00,  9.02it/s]


finished batch of 0-300. Going to wait 27.72s


100%|██████████| 300/300 [00:36<00:00,  8.30it/s]


finished batch of 0-300. Going to wait 24.85s


100%|██████████| 300/300 [00:39<00:00,  7.66it/s]


...

100%|██████████| 175/175 [00:48<00:00,  3.64it/s]


finished batch of 2700-3000. Going to wait 12.87s


100%|██████████| 175/175 [00:18<00:00,  9.52it/s]


finished batch of 2700-3000. Going to wait 42.62s


,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
0,000001.SZ,20190307,20181231,交通银行-易方达50指数证券投资基金,4.414380e+07,0.26
1,000001.SZ,20190307,20181231,中国平安保险(集团)股份有限公司-集团本级-自有资金,8.510493e+09,49.56
2,000001.SZ,20190307,20181231,中国平安人寿保险股份有限公司-自有资金,1.049463e+09,6.11
3,000001.SZ,20190307,20181231,香港中央结算有限公司(陆股通),4.307515e+08,2.51
4,000001.SZ,20190307,20181231,中国证券金融股份有限公司,4.292327e+08,2.50
...,...,...,...,...,...,...
488489,603999.SH,20190426,20190331,易方达基金-工商银行-易方达基金臻选2号资产管理计划,2.825600e+06,0.49
488490,603999.SH,20190426,20190331,时代出版传媒股份有限公司,3.148200e+06,0.55
488491,603999.SH,20190426,20190331,外研投资发展(北京)有限公司,3.470270e+06,0.60
488492,603999.SH,20190426,20190331,全国社会保障基金理事会转持二户,6.694056e+06,1.16


In [25]:
async def get_fina_mainbz(code):
    return None if code == '' else pro.fina_mainbz_vip(ts_code=code)
async def get_fina_mainbz_first_half(code, year):
    return None if code == '' else pro.fina_mainbz_vip(ts_code=code, start_date=f'20{year}0101', end_date=f'20{year}0630')
async def get_fina_mainbz_second_half(code):
    return None if code == '' else pro.fina_mainbz_vip(ts_code=code, start_date=f'20{year}0701', end_date=f'20{year}1231')

if not recrawl:
    fina_mainbz = pd.read_csv('fina_mainbz.csv')
else:
    fina_mainbz = []
    request_per_minute = 300
    for i in range(0, len(stock_info_list.ts_code), request_per_minute):
        st = time.time()
        aync_tasks = [asyncio.create_task(get_fina_mainbz(code)) for code in stock_info_list.ts_code[i:i+request_per_minute]]
        fina_mainbz.extend([await t for t in tqdm(asyncio.as_completed(aync_tasks), total=len(aync_tasks))])
        et = time.time()
        print(f'finished batch of {i}-{i+request_per_minute}. Going to wait {(st+61-et):.2f}s')
        time.sleep(max(st+61-et, 0))
        
    fina_mainbz = pd.concat(fina_mainbz, ignore_index=True)
    fina_mainbz.to_csv('fina_mainbz.csv')
    
display(fina_mainbz)

100%|██████████| 300/300 [00:39<00:00,  7.58it/s]


finished batch of 0-300. Going to wait 21.40s


100%|██████████| 300/300 [00:38<00:00,  7.84it/s]


finished batch of 300-600. Going to wait 22.73s


100%|██████████| 300/300 [00:36<00:00,  8.12it/s]


...

100%|██████████| 300/300 [00:33<00:00,  8.99it/s]


finished batch of 2400-2700. Going to wait 27.63s


100%|██████████| 175/175 [00:19<00:00,  9.04it/s]


finished batch of 2700-3000. Going to wait 41.65s


,ts_code,end_date,bz_item,bz_sales,bz_profit,bz_cost,curr_type
0,000001.SZ,20201231,发放贷款及垫款,1.444150e+11,NaN,NaN,CNY
1,000001.SZ,20201231,利息收入:金融投资利息收入,3.154300e+10,NaN,NaN,CNY
2,000001.SZ,20201231,金融企业往来,7.850000e+09,NaN,NaN,CNY
3,000001.SZ,20201231,存放中央银行款项,3.379000e+09,NaN,NaN,CNY
4,000001.SZ,20201231,非利息收入:手续费及佣金收入:代理委托业务(产品),9.426000e+09,NaN,NaN,CNY
...,...,...,...,...,...,...,...
1010023,603999.SH,20111231,期刊,2.145086e+08,86766000.0,127742600.0,CNY
1010024,603999.SH,20111231,新媒体及其他,9.431300e+06,2735400.0,6695900.0,CNY
1010025,603999.SH,20111231,西北,3.495706e+08,NaN,NaN,CNY
1010026,603999.SH,20111231,东北,1.277670e+07,NaN,NaN,CNY
